# Invisible Light - plotting a galaxy's colours

In this notebook, you will be able to use data from the LOFAR deep fields release to choose a galaxy and plot its _spectral energy distribution_ (SED)- that is, how bright the galaxy is across the electromagnetic spectrum. This data is from a well-studied field called the Lockman Hole, which is a patch of sky located in the constellation of Ursa Major. 

First we will import the Python libraries that we will use in this exercise:


In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt

Then we will import the data files. All of the data is publicly available on the LOFAR surveys website, but the data files are very large (> 1 GB!) so I'm using a short version of the table that I put together for my research:


In [2]:
# image = fits.open("https://www.lofar-surveys.org/public/deepfields/data_release/lockman/lockman_radio_image.fits")[0].data

data_file = fits.open("https://star.herts.ac.uk/~joanna/master_table.fits")
table = data_file[1].data
headers = data_file[1].header

This data file is a table of data, with fluxes at lots of different wavelengths across the electromagnetic spectrum, from low frequency radio in LOFAR all the way through to optical and UV.


Each row of the table is a different galaxy, and you can find its position on the sky by looking at the right ascension (RA) and declination (Dec) in the table, which are given with the keys 'RA_1' and 'Dec_1':




In [21]:
n = 6
print(f"Coordinates of galaxy {n}: {table[n-1]['RA_1']:0.03f}, {table[n-1]['Dec_1']:0.03f}")

Coordinates of galaxy 6: 161.480, 59.258


<!-- "ugriz" is short for U-band, G-band, R-band, I-band, and Z-band, detailed on [the Wikipedia article on photometric systems](https://en.wikipedia.org/wiki/Photometric_system): -->
$$\begin{array}{|c|c|c|}
\hline \text{Band}& \text{Table key} & \text{Wavelength}(\lambda)\\
\hline \text{u}   & \text{'u_flux'}        & \text{365 nm}\\
\hline \text{g}   & \text{'g_flux'}        & \text{475 nm}\\
\hline \text{r}   & \text{'r_flux'}        &  \text{658 nm}\\
\hline \text{i}   & \text{'i_rcs_flux'}        &  \text{806 nm}\\
\hline \text{z}   & \text{'z_flux'}        &  \text{900 nm}\\
\hline
\end{array}$$

In [13]:
galaxy = table[n-1]
print(galaxy['z_flux'])


-0.30688596
